In [ ]:
import pygsheets
import pandas as pd
import altair as alt

alt.renderers.enable('default')

In [ ]:
gc = pygsheets.authorize()

In [ ]:
sht1 = gc.open_by_key('1QijK93G8_BdT4n7aLs9IEY9d-lNko4sy-UBIDf6kzw0').worksheet_by_title("Sheet1")

In [ ]:
df = sht1.get_as_df()

df.drop(0, inplace=True)

df = df[0:65]

df.rename(columns={
    "Discipline (d'après site France Culture)": "discipline",
    "Genre H ?": "genre"
}, inplace=True)

df["nom"] = df.apply(lambda r: r["Prénom"]+" "+r["Nom"] , axis=1)

df.head()

In [ ]:
df.shape

In [ ]:
df["Âge"] = pd.to_numeric(df["Âge"])

In [ ]:
color_scale = alt.Scale(domain=["F", "H"], range=["#FF851B", "#3D9970"])

chart = alt.Chart(df[["nom", "Âge", "genre", "discipline"]])

bars = chart.mark_bar().encode(
    y= alt.Y("nom:N", sort=alt.EncodingSortField(field="Âge", op="sum")),
    x="Âge:Q",
    color=alt.Color("genre", scale=color_scale)
)

text = bars.mark_text(
    align="left",
    color="black",
    baseline="middle",
    dx=3
).encode(
    text="Âge:Q"
)

(bars).configure(background="white")

In [ ]:
bars = chart.mark_bar().encode(
    y = alt.Y("discipline:N", sort=alt.EncodingSortField(field="discipline", op="count")),
    x = "count(*):Q",
    color=alt.Color("genre", scale=color_scale),
    order=alt.Order("genre", sort="ascending")
)

(bars).configure(background="white")

In [ ]:
bars = chart.mark_text(size=24, baseline="middle").encode(
    y = alt.Y("genre:O", axis=None),
    x = alt.X("n:O", axis=None),
    text= "emoji:N",
    row= alt.Row('discipline:N',
                 sort=alt.EncodingSortField(field="nom", op="count"),
                 header=alt.Header(labelAngle=1, title=""))
).transform_calculate(
    emoji="{'F':'👩🏻', 'H':'👨🏻'}[datum.genre]"
).transform_window(
    n = 'rank()',
    groupby=['discipline', 'genre']
).properties(width=400, height=75)

(bars).configure(background="white")

In [ ]:
bars = chart.mark_bar().encode(
    y = alt.Y("discipline:N", sort=alt.EncodingSortField(field="Âge", op="mean")),
    x = "mean(Âge):Q"
)

(bars).configure(background="white")

In [ ]:
bars = chart.mark_bar().encode(
    y = alt.Y("genre:N", title=None),
    x = "mean(Âge):Q",
    color = alt.Color("genre", scale=color_scale),
    row = alt.Row("discipline:N", sort=alt.EncodingSortField(field="Âge", op="mean"), header=alt.Header(labelAngle=1))
)

(bars).configure(background="white")